In [ ]:
! git clone https://github.com/OpenMined/PySyft
! cd PySyft && pip install -r pip-dep/requirements.txt
! cd PySyft && pip install -r pip-dep/requirements_udacity.txt
! cd PySyft && python setup.py install

Cloning into 'PySyft'...
remote: Enumerating objects: 44013, done.
remote: Total 44013 (delta 0), reused 0 (delta 0), pack-reused 44013
Receiving objects: 100% (44013/44013), 40.33 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (30301/30301), done.
     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 2.0MB 10.6MB/s 
     |████████████████████████████████| 1.8MB 41.1MB/s 
     |████████████████████████████████| 2.1MB 37.1MB/s 
     |████████████████████████████████| 450kB 42.1MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 4.0MB 34.5MB/s 
     |████████████████████████████████| 753.4MB 25kB/s 
     |████████████████████████████████| 204kB 33.3MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 9.0MB 18.6MB/s 
     |██████████████████████████

In [ ]:
! pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

  Cloning https://github.com/PytorchLightning/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-u91wya78
  Running command git clone -q https://github.com/PytorchLightning/pytorch-lightning.git /tmp/pip-req-build-u91wya78
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.8.4-cp36-none-any.whl size=305522 sha256=3b9bda671ca20d3eb436687d917567c5e04daaebbc95a8dccb2326779286ace7
  Stored in directory: /tmp/pip-ephem-wheel-cache-a35pf4w8/wheels/71/30/98/7d39a914d2f5563909745100a78950c34221690e6fe574a915
Successfully built pytorch-lightning
  Found existing installation: pytorch-lightning 0.8.4
    Uninstalling pytorch-lightning-0.8.4:
      Successfully uninstalled pytorch-lightning-0.8.4


Based on Tutorial Part 6 https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2006%20-%20Federated%20Learning%20on%20MNIST%20using%20a%20CNN.ipynb

**RESTART RUNTIME**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [ ]:
epochs = 10

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

This is where new code comes in:

In [ ]:
from pytorch_lightning.core.lightning import LightningModule

class LNet(LightningModule):
    def __init__(self):
        super(LNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        data, target = batch
        self.send(data.location)
        output = self.forward(data)
        loss = F.nll_loss(output, target).get()
        self.get()
        return {'loss': loss }
    
    def validation_step(self, batch, batch_idx):
        data, target = batch
        #self.send(data.location)
        output = self.forward(data)
        test_loss = F.nll_loss(output, target, reduction='sum')
        pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
        correct = pred.eq(target.view_as(pred)).sum()
        #self.get()
        return {'val_loss': test_loss, 'correct': correct }

    def train_dataloader(self):
       return sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
            datasets.MNIST('../data', train=True, download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ]))
            .federate((bob, alice))
        )
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
            batch_size=args.test_batch_size, shuffle=False, **kwargs)

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'val_loss': avg_loss }
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=args.lr)
        return optimizer

In [ ]:
from pytorch_lightning import Trainer

net = LNet().to(device)
trainer = Trainer(gpus=1, max_epochs=2, progress_bar_refresh_rate=20)
trainer.fit(net)

# vaidation tqdm progress bar looks like 0/2 because progress_bar_refresh_rate=20
# without that param, tqdm progress bar refreshes too much and fills browser RAM
# https://github.com/PyTorchLightning/pytorch-lightning/issues/1403
# https://github.com/PyTorchLightning/pytorch-lightning/issues/2419

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv2d | 520   
1 | conv2 | Conv2d | 25 K  
2 | fc1   | Linear | 400 K 
3 | fc2   | Linear | 5 K   
INFO:lightning:
  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv2d | 520   
1 | conv2 | Conv2d | 25 K  
2 | fc1   | Linear | 400 K 
3 | fc2   | Linear | 5 K   
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have

1

In [ ]:
torch.save(net.state_dict(), "mnist_cnn.pt")